In [0]:
def extract_data_contract_metadata(data_contract: dict) -> list:
    """
    Extracts the high-level metadata fields from an ODCS data contract.
    Args:
        contract (dict): The loaded YAML data contract.
    Returns:
        list: A single-item list containing a dictionary with extracted fields.
    """
    try:
        result = {
            "name": data_contract.get("name"),
            "apiversion": data_contract.get("apiVersion"),  # note lowercase 'v'
            "kind": data_contract.get("kind"),
            "id": data_contract.get("id"),
            "version": data_contract.get("version"),
            "domain": data_contract.get("domain"),
            "status": data_contract.get("status"),
            "dataproduct": data_contract.get("dataProduct"),
            "tenant": data_contract.get("tenant"),
            "description": data_contract.get("description", {}),
            "tags": data_contract.get("tags", []),
        }
        return [result]
    except: return None

In [0]:
def extract_quality_rules_metadata(data_contract: dict) -> dict:
    """
    Extracts and cleans the quality rules from each table in a given data contract schema.
    Args:
        data_contract (dict): The full ODCS data contract containing a "schema" block.
    Returns:
        dict: A dictionary mapping each table name to its quality rules.
              Format: { "table_name": { "quality": [ ... ] }, ... }
    """
    try:
        quality_dict = {}
        for table in data_contract["schema"]:
            table_name = table.get("name")
            quality_rules = table.get("quality", [])

            if not quality_rules:
                continue

            formatted_quality = []
            for rule in quality_rules:
                rule_copy = rule.copy()
                if "query" in rule_copy:
                    rule_copy["query"] = rule_copy["query"].strip()
                formatted_quality.append(rule_copy)

            quality_dict[table_name] = {"quality": formatted_quality}
        return quality_dict
    except: return None

In [0]:
def extract_sla_metadata(data_contract: dict) -> list:
    """
    Extracts SLA metadata from an ODCS data contract.
    Args:
        contract (dict): The loaded YAML or JSON data contract.
    Returns:
        list: A list of SLA property dictionaries with standardized keys.
    """
    try:
        sla_list = data_contract.get("slaProperties", [])
        formatted_sla = []

        for item in sla_list:
            sla = {
                "property": item.get("property", ""),
                "value": item.get("value"),
                "unit": item.get("unity", ""),  # unit was "unity" in the contract
                "element": item.get("element", "")
            }
            # Optionally include valueExt and driver if present
            if "valueExt" in item:
                sla["valueExt"] = item["valueExt"]
            if "driver" in item:
                sla["driver"] = item["driver"]
            formatted_sla.append(sla)

        return formatted_sla
    except: return None